In [1]:
import pandas as pd
import os
import numpy as np

In [33]:
#load up all the data
inputfolder = 'data'
filelist = os.listdir(inputfolder)
framedict = dict()
for file in filelist:
    filename,ext = os.path.splitext(file)
    pathway = os.path.join(inputfolder,file)
    tempdf = pd.read_csv(pathway)
    framedict[filename] = tempdf
    del tempdf
dictnames = [x for x in framedict.keys()]
print(dictnames)

['ATP_matches', 'aus_open_men_match_odds', 'aus_open_men_tournament_odds', 'aus_open_women_match_odds', 'aus_open_women_tournament_odds', 'WTA_matches']


C:\Users\Hugh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,12,13,14,17,18,24,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Looking at the mens data

In [45]:
#load mens dataframes into variables and inspect data
ATPframe = framedict['ATP_matches'].copy() #we copy so we can run the code multiple times without errors, not really necessary
AOMensMatchOdds = framedict[ 'aus_open_men_match_odds'].copy()
AOMensTournOdds = framedict['aus_open_men_tournament_odds'].copy()
display(ATPframe.head())
display(AOMensMatchOdds.head())
display(AOMensTournOdds.head())

#initial cleaning
#checking that we're only looking at mens data
print(AOMensMatchOdds['men_women'].unique())
print(AOMensTournOdds['men_women'].unique())

#only mens, so we can drop this column
AOMensMatchOdds.drop('men_women',axis=1,inplace=True)
AOMensTournOdds.drop('men_women',axis=1,inplace=True)

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,2-Jan-12,Hard,First Round,106,97,0,2,...,3,21,33,13,26,1,3,15,49,49
1,Dudi Sela,Fabio Fognini,Chennai,2-Jan-12,Hard,First Round,83,48,0,2,...,4,17,32,5,26,0,1,8,33,30
2,Go Soeda,Frederico Gil,Chennai,2-Jan-12,Hard,First Round,120,102,0,2,...,2,45,70,18,35,2,4,36,103,99
3,Yuki Bhambri,Karol Beck,Chennai,2-Jan-12,Hard,First Round,345,101,0,2,...,1,15,33,13,29,2,3,15,46,43
4,Yuichi Sugita,Olivier Rochus,Chennai,2-Jan-12,Hard,First Round,235,67,0,2,...,0,19,32,13,22,1,7,30,78,62


,tournament,men_women,match_date,player_1,player_1_odds,player_2,player_2_odds
0,Australian Open 2012,Men,2012-01-17,Potito Starace,1.77,Tatsuma Ito,2.24
1,Australian Open 2012,Men,2012-01-17,Nicolas Mahut,5.30,Radek Stepanek,1.20
2,Australian Open 2012,Men,2012-01-17,Filippo Volandri,40.00,Milos Raonic,1.02
3,Australian Open 2012,Men,2012-01-17,Lukas Rosol,3.25,Philipp Petzschner,1.42
4,Australian Open 2012,Men,2012-01-17,Cedrik Marcel Stebe,3.60,Lleyton Hewitt,1.36


,tournament,men_women,player,odds
0,Australian Open 2012,Men,Novak Djokovic,2.46
1,Australian Open 2012,Men,Roger Federer,5.60
2,Australian Open 2012,Men,Andy Murray,7.60
3,Australian Open 2012,Men,Rafael Nadal,9.00
4,Australian Open 2012,Men,Jo Wilfried Tsonga,22.00


['Men']
['Men']


In [55]:
#convert dates to datetime
ATPframe['datetime'] = pd.to_datetime(ATPframe['Tournament_Date'])
AOMensMatchOdds['datetime'] = pd.to_datetime(AOMensMatchOdds['match_date'])
display(AOMensMatchOdds.loc[:,['match_date','datetime']].head())

#testing splitting up dates into day/month/year
AOMensMatchOdds['year'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).year
AOMensMatchOdds['month'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).month
AOMensMatchOdds['day'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).day
ATPframe['year'] = pd.DatetimeIndex(ATPframe['datetime']).year
ATPframe['month'] = pd.DatetimeIndex(ATPframe['datetime']).month
ATPframe['day'] = pd.DatetimeIndex(ATPframe['datetime']).day
display(ATPframe.loc[:,['year','month','day','datetime']].head())

#we only have odds data for Australian Open, so we're gonna just grab ATP data for Australian Opens
print(AOMensTournOdds['tournament'].unique())
print(AOMensTournOdds['tournament'].unique() == AOMensMatchOdds['tournament'].unique())#checking to see if odds data are from the same tournaments

#new column for Aus open names so we can match it
def AusOpen(row):
    tourney = row['Tournament']
    if 'Australian Open' in tourney:
        return 'Ausralian Open '+str(row['year'])
    else:
        return np.nan
ATPframe['AusOpen'] = ATPframe.apply(AusOpen,axis=1)
print(ATPframe['AusOpen'].unique())
AusOpenframe = ATPframe.dropna(subset =['AusOpen'],axis=0)
display(AusOpenframe.head())

,match_date,datetime
0,2012-01-17,2012-01-17
1,2012-01-17,2012-01-17
2,2012-01-17,2012-01-17
3,2012-01-17,2012-01-17
4,2012-01-17,2012-01-17


,year,month,day,datetime
0,2012,1,2,2012-01-02
1,2012,1,2,2012-01-02
2,2012,1,2,2012-01-02
3,2012,1,2,2012-01-02
4,2012,1,2,2012-01-02


['Australian Open 2012' 'Australian Open 2013' 'Australian Open 2014'
 'Australian Open 2015' 'Australian Open 2016' 'Australian Open 2017'
 'Australian Open 2018']
[ True  True  True  True  True  True  True]
[nan 'Ausralian Open 2012' 'Ausralian Open 2013' 'Ausralian Open 2014'
 'Ausralian Open 2015' 'Ausralian Open 2016' 'Ausralian Open 2017'
 'Ausralian Open 2018']


,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won,datetime,year,month,day,AusOpen
137,Adrian Menendez-Maceiras,Alejandro Gonzalez,"Australian Open, Melbourne",16-Jan-12,Hard,Qualifying,195,270,0,2,...,3,10,24,53,44,2012-01-16,2012,1,16,Ausralian Open 2012
138,Marco Crugnola,Amir Weintraub,"Australian Open, Melbourne",16-Jan-12,Hard,Qualifying,274,220,0,2,...,1,1,14,43,46,2012-01-16,2012,1,16,Ausralian Open 2012
139,Sergei Bubka,Andre Begemann,"Australian Open, Melbourne",16-Jan-12,Hard,Qualifying,152,227,0,2,...,3,3,30,76,82,2012-01-16,2012,1,16,Ausralian Open 2012
140,Denys Molchanov,Andrew Harris,"Australian Open, Melbourne",16-Jan-12,Hard,Qualifying,267,.,0,2,...,1,2,21,64,57,2012-01-16,2012,1,16,Ausralian Open 2012
141,Bjorn Phau,Arnaud Clement,"Australian Open, Melbourne",16-Jan-12,Hard,Qualifying,174,157,0,2,...,3,10,32,73,63,2012-01-16,2012,1,16,Ausralian Open 2012
